<a href="https://colab.research.google.com/github/Manvikaul/corona-chatbot/blob/master/chatbot_improved.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
pip install nltk

In [3]:
pip install newspaper3k

     |████████████████████████████████| 215kB 6.6MB/s 
     |████████████████████████████████| 194kB 18.3MB/s 
     |████████████████████████████████| 51kB 7.3MB/s 
     |████████████████████████████████| 7.4MB 18.8MB/s 
  Created wheel for feedparser: filename=feedparser-5.2.1-cp36-none-any.whl size=44940 sha256=5cf017c6884f2532081a7f46a5d4acfbac60f0533d0ea7cfa42bab37089ddea1
  Stored in directory: /root/.cache/pip/wheels/8c/69/b7/f52763c41c5471df57703a0ef718a32a5e81ee35dcf6d4f97f
  Created wheel for feedfinder2: filename=feedfinder2-0.0.4-cp36-none-any.whl size=3357 sha256=a6760229b0c3cffdc13c244bdc12e024e94eb89de195f6b4b2ffe887132d8d3c
  Stored in directory: /root/.cache/pip/wheels/de/03/ca/778e3a7a627e3d98836cc890e7cb40c7575424cfd3340f40ed
  Created wheel for jieba3k: filename=jieba3k-0.35.1-cp36-none-any.whl size=7398406 sha256=b69a47a5b7b5e328acf2180f9be248762a63c398cacbe6de6518850a30982574
  Stored in directory: /root/.cache/pip/wheels/83/15/9c/a3f1f67e7f7181170ad37d32e503c35da2

In [0]:
#IMPORTS
from newspaper import Article
import random
import string
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
import nltk
import numpy as np
import warnings

In [0]:
#Ignoring warnings
warnings.filterwarnings('ignore')

In [6]:
nltk.download('punkt')
nltk.download('wordnet')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Unzipping corpora/wordnet.zip.


True

In [7]:
article=Article('https://www.webmd.com/lung/coronavirus')
article.download()
article.parse()
article.nlp()

corpus=article.text

print(corpus)

Editor's note: For the latest updates on the 2020 coronavirus outbreak, see our news coverage .

What Is COVID-19?

A coronavirus is a kind of common virus that causes an infection in your nose, sinuses, or upper throat. Most coronaviruses aren't dangerous.

COVID-19 is a disease that can cause what doctors call a respiratory tract infection. It can affect your upper respiratory tract (sinuses, nose, and throat) or lower respiratory tract (windpipe and lungs). It's caused by a coronavirus named SARS-CoV-2.

It spreads the same way other coronaviruses do, mainly through person-to-person contact. Infections range from mild to serious.

SARS-CoV-2 is one of seven types of coronavirus, including the ones that cause severe diseases like Middle East respiratory syndrome (MERS) and sudden acute respiratory syndrome (SARS). The other coronaviruses cause most of the colds that affect us during the year but aren’t a serious threat for otherwise healthy people.

In early 2020, after a December 20

In [8]:
text=corpus
sent_tokens=nltk.sent_tokenize(text)

print(sent_tokens)

["Editor's note: For the latest updates on the 2020 coronavirus outbreak, see our news coverage .", 'What Is COVID-19?', 'A coronavirus is a kind of common virus that causes an infection in your nose, sinuses, or upper throat.', "Most coronaviruses aren't dangerous.", 'COVID-19 is a disease that can cause what doctors call a respiratory tract infection.', 'It can affect your upper respiratory tract (sinuses, nose, and throat) or lower respiratory tract (windpipe and lungs).', "It's caused by a coronavirus named SARS-CoV-2.", 'It spreads the same way other coronaviruses do, mainly through person-to-person contact.', 'Infections range from mild to serious.', 'SARS-CoV-2 is one of seven types of coronavirus, including the ones that cause severe diseases like Middle East respiratory syndrome (MERS) and sudden acute respiratory syndrome (SARS).', 'The other coronaviruses cause most of the colds that affect us during the year but aren’t a serious threat for otherwise healthy people.', 'In ea

In [9]:
remove_punct_dict=dict((ord(punct),None) for punct in string.punctuation)

print(string.punctuation)

!"#$%&'()*+,-./:;<=>?@[\]^_`{|}~


In [10]:
print(remove_punct_dict)

{33: None, 34: None, 35: None, 36: None, 37: None, 38: None, 39: None, 40: None, 41: None, 42: None, 43: None, 44: None, 45: None, 46: None, 47: None, 58: None, 59: None, 60: None, 61: None, 62: None, 63: None, 64: None, 91: None, 92: None, 93: None, 94: None, 95: None, 96: None, 123: None, 124: None, 125: None, 126: None}


In [13]:
def LemNormalize(text):
  return nltk.word_tokenize(text.lower().translate(remove_punct_dict))

print(LemNormalize(text))

['editors', 'note', 'for', 'the', 'latest', 'updates', 'on', 'the', '2020', 'coronavirus', 'outbreak', 'see', 'our', 'news', 'coverage', 'what', 'is', 'covid19', 'a', 'coronavirus', 'is', 'a', 'kind', 'of', 'common', 'virus', 'that', 'causes', 'an', 'infection', 'in', 'your', 'nose', 'sinuses', 'or', 'upper', 'throat', 'most', 'coronaviruses', 'arent', 'dangerous', 'covid19', 'is', 'a', 'disease', 'that', 'can', 'cause', 'what', 'doctors', 'call', 'a', 'respiratory', 'tract', 'infection', 'it', 'can', 'affect', 'your', 'upper', 'respiratory', 'tract', 'sinuses', 'nose', 'and', 'throat', 'or', 'lower', 'respiratory', 'tract', 'windpipe', 'and', 'lungs', 'its', 'caused', 'by', 'a', 'coronavirus', 'named', 'sarscov2', 'it', 'spreads', 'the', 'same', 'way', 'other', 'coronaviruses', 'do', 'mainly', 'through', 'persontoperson', 'contact', 'infections', 'range', 'from', 'mild', 'to', 'serious', 'sarscov2', 'is', 'one', 'of', 'seven', 'types', 'of', 'coronavirus', 'including', 'the', 'ones', 

In [0]:
GREETING_INPUTS=['hey','hi','hello','heya','howdy','whatsup','wassup','hola']
GREETING_OUTPUTS=['hey!','hello!','heya!','howdy!','hey there!','holaa!']
